In [1]:
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
import requests

import cv2
import matplotlib.pyplot as plt

import math
import numpy as np
from scipy.ndimage import imread

import sys
import time
import os
from copy import deepcopy
import ctypes

In [6]:
HWC = (360, 480, 3)
WH = (480, 360)
IP = "54.180.93.44"
DB_SERVER_IP = "http://" + IP + ":3000/auth"
KAFKA_SERVER_IP = "http://" + IP + ":3000/stream"

###############################################################################################################
class BeforeLogin(QWidget):
    def __init__(self, path=['haarcascades/haarcascade_frontalface_default.xml', 'haarcascades/haarcascade_eye.xml']):
        QWindow.__init__(self)
        self.path = path
        self.token = False
        
        self.central_layout = QGridLayout()
        self.central_widget = QStackedWidget()
        self.central_layout.addWidget(self.central_widget)
        
        lst = ["ID  : ", "PW : "]
        self.label_lst, self.edit_lst = self.making_Label_Edit(lst)
        pos_lst = [(1,1), (2,1)]
        
        layout = QGridLayout() 
        LoginFrame = self.making_Frame(layout, self.label_lst, self.edit_lst, pos_lst)
        LoginFrame.setFrameShape(QFrame.StyledPanel)
        
        LoginButton = QPushButton("로그인 하기")
        LoginButton.clicked.connect(self.btnOkClicked)
        
        LoginLayout = QGridLayout() 
        LoginLayout.addWidget(LoginFrame)
        LoginLayout.addWidget(LoginButton)
        
        coveringWidget = QWidget()
        coveringWidget.setLayout(LoginLayout)
        self.central_widget.addWidget(coveringWidget)
        self.setLayout(self.central_layout)
        
        print("#"*60)
        print("[{}] : program start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("#"*60, "\n")
        
    def btnOkClicked(self):
        ID = self.edit_lst[0].text().strip()
        PW = self.edit_lst[1].text().strip()
        
        self.login(ID, PW)

    def making_Label_Edit(self, name_lst) :
        label_lst = []
        edit_lst = []
        
        for name in name_lst :
            label = QLabel(name)
            edit = QLineEdit()
            
            label_lst.append(label)
            edit_lst.append(edit)
        
        return label_lst, edit_lst

    def making_Frame(self, layout, label_lst, widget_lst, pos_lst) :
        for label, widget, pos in zip(label_lst, widget_lst, pos_lst) :  
            layout.addWidget(label, pos[0], pos[1])
            layout.addWidget(widget, pos[0], pos[1]+1)
            
        frame = QFrame()
        frame.setLayout(layout)
        
        return frame
    
    def login(self, ID, PW) :
        print("#"*60)
        print("[{}] : '{}' try to login".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time())), ID))
        
        if self.checkUserDB(ID,PW) :
            print("[{}] : '{}' success to login".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time())), ID))
            print("#"*60, "\n")
            
            AfterLayout = self.AfterLogin()
            AfterLayout.setToken(self.token)
            self.central_widget.addWidget(AfterLayout)
            self.central_widget.setCurrentWidget(AfterLayout)
        
        else :
            print("[{}] : '{}' fail to login".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time())), ID))
            print("#"*60, "\n")
            
            QMessageBox.information(self, "Login failed", "check ID & PW")
        
    def AfterLogin(self) :
        self.afterLoginWidget = AfterLoginWidget(self.token)
        return self.afterLoginWidget
    
    def checkUserDB(self, ID, PW, host=DB_SERVER_IP) :
        data = {"email_id" : ID, "password" : PW}
        result = requests.post(host, data=data)
        
        if "success" in result.json() :
            self.token = result.json()["token"]
            
            with open("./data/register/registered_info/token.txt", "w") as f:
                f.write(self.token)
            
            return True
        else :
            return False
###############################################################################################################


###############################################################################################################
class AfterLoginWidget(QWidget):
    def __init__(self, token, path=['haarcascades/haarcascade_frontalface_default.xml', 'haarcascades/haarcascade_eye.xml'], interval=3, parent=None):
        super().__init__(parent)

        self.interval = interval
        self.path = path
        self.start = False
        self.flag_while = False
        self.global_flag = [True]
        self.isDialog = False
        self.dialog_size = (0,0)
                
        layout = QVBoxLayout()
        self.msg = QLabel("ready for detection")
        self.ready_detection()

        layout = QVBoxLayout()
        self.RegisterButton = QPushButton('Register')
        self.RunButton = QPushButton('Start')
        self.ExitButton = QPushButton('Stop')

        self.RegisterButton.clicked.connect(self.registerClicked)
        self.RunButton.clicked.connect(self.runClicked)
        self.ExitButton.clicked.connect(self.stopClicked)

        layout.addWidget(self.RegisterButton)
        layout.addWidget(self.msg)
        layout.addWidget(self.RunButton)
        layout.addWidget(self.ExitButton)
        self.setLayout(layout)
        
        self.detectionThread = detectionThread(self.global_flag)
        
    def setToken(self, token) :
        self.token = token
        self.detectionThread.setToken(token)
        
    def registerClicked(self) :
        if (not self.start) and self.flag_while:
            params = [self.video_capture, self.face_cascade, self.eyes_cascade, 
                      self.msg, self.interval, self.flag_while]
            self.openRegisterThread = openRegisterThread(params, self)
            self.openRegisterThread.start()
            
            if self.dialog_size[0] !=0 :
                print("#"*60)
                print("[{}] : try to register".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                print("dialog size has {}".format(self.dialog_size))
                print("#"*60, "\n")
                self.registerDialog = registerDialog(params, self.dialog_size, self)
                self.registerDialog.show()
                self.dialog_size = (0,0)
                self.isDialog = True
                
            else :
                print("#"*60)
                print("[{}] : fail to register -- ready for register".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                print("#"*60, "\n")
                QMessageBox.information(self, "Register failed", "ready for register, try again")
        
        else : 
            print("#"*60)
            print("#"*60, "\n")
            QMessageBox.information(self, "Register failed", "stop the detection first")
        
    def runClicked(self) :
        self.global_flag[0] = True

        if (not self.start) and self.flag_while:
            self.start = True
            
            print("#"*60)
            print("[{}] : detection start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
            print("#"*60, "\n")
            
            
            params = [self.video_capture, self.face_cascade, self.eyes_cascade, 
                      self.msg, self.interval, self.flag_while]

            self.detectionThread.setParams(params)
            self.detectionThread.setRecognizer(self.recognizer)
            self.detectionThread.start()
    
    def stopClicked(self) : 
        self.global_flag[0] = False
        
        if self.start :
            self.start = False

            print("")
            print("#"*60)
            print("[{}] : detection stopped".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
            print("#"*60, "\n")

            self.msg.setText("ready for detection")
 
    def ready_detection(self) : 
        self.face_cascade = cv2.CascadeClassifier(self.path[0])
        self.eyes_cascade = cv2.CascadeClassifier(self.path[1])
        
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()
            
        self.video_capture = cv2.VideoCapture(0)
        _, first_frame = self.video_capture.read()

        print("#"*60)
        try : 
            if not first_frame :
                print("[{}] : camera is not conntected".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                self.flag_while = False
        except :
            print("[{}] : camera is conntected".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
            self.flag_while = True
        print("#"*60, "\n")
###############################################################################################################


###############################################################################################################
class detectionThread(QThread) :
    def __init__(self, global_flag):
        QThread.__init__(self)
        self.global_idx = 0
        self.global_flag = global_flag
        
        self.img_count = 0
        self.face_count = 0
        self.eye_count = 0
        self.confidence = []
        self.coord = []
        
        self.change_cnt = 0
        self.change_conf_lst = []
        self.yml_ver = 1
        self.rd_flag = False
        
        self.sThread = saveThread()
        self.rdThread = reg_detThread()

    def __del__(self):
        self.flag_while = False
        self.wait()
        
    def setParams(self, params) :
        self.video_capture = params[0]
        self.face_cascade = params[1]
        self.eyes_cascade = params[2]
        self.msg = params[3]
        self.interval = params[4]
        self.flag_while = params[5]
        
    def setRecognizer(self, recognizer) :
        self.recognizer = recognizer
        
    def setToken(self, token) :
        self.token = token
        self.sThread.setToken(token)
        
    def mean(self, lst) :
        if len(lst) == 0 :
            return 0
        
        return sum(lst) / len(lst)

    def run(self):
        self.face_detection()
    
    def face_detection(self) :
        self.recognizer.read("./data/register/registered_info/train1.yml")
        start_time = time.time()
        append_cnt = 0
        
        label = 0
        confidence = 0
        
        # 재학습용 param
        re_img_cnt = 0
        re_face_cnt = 0
        re_eye_cnt = 0
        re_train_img_cnt = []
        re_train_face_cnt = []
        re_train_eye_cnt = []
        
        # detection
        while self.flag_while and self.global_flag[0]:
            read, frame = self.video_capture.read()
            frame = cv2.resize(frame, WH)

            if not read :
                continue
            
            if self.rd_flag :
                re_img_cnt += 1
                
            self.img_count +=  1    
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            flag, detected_frame, coords = self.detect(gray, frame)
            
            # 얼굴이 인식되지 않았을 경우
            if not flag[0] :
                #print("[{}] : (1) face and eye are not detected".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                self.msg.setText("(msg1) face and eye are not detected")
                self.face_count += 0
            
            # 얼굴이 인식되었을 경우
            else :
                self.face_count += 1
                label, confidence = self.recognizer.predict(detected_frame)
                coord = self.size_filter(coords[0])
                self.confidence.append(confidence)
                self.coord.append(coord[0])
                
                if self.rd_flag :
                    re_face_cnt += 1
                
                # 눈이 인식되지 않았을 경우
                if not flag[1] :
                    #print("[{}] : (2) face is detected, but eye is not detected, --confidence {:.1f}".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time())), confidence))
                    self.msg.setText("(msg2) face is detected, but eye is not detected")
                    self.eye_count += 0
                
                # 눈이 인식되었을 경우
                else : 
                    #print("[{}] : (3) face and eye are detected, --confidence {:.1f}".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time())), confidence))
                    self.msg.setText("(msg3) face and eye are detected")
                    self.eye_count += 1
                    
                    if self.rd_flag :
                        re_eye_cnt += 1
                
                # 재학습되는 경우
                if self.rd_flag :
                    self.rdThread.add(detected_frame, coord[0])
                    
                    if self.rdThread.data_size == 200 :
                        print("-"*90)
                        print("[{}] : re-train starts".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        print("-"*90)
                        
                        self.rdThread.setVersion(self.yml_ver)
                        self.rdThread.setDThread(self)
                        self.rdThread.setCnt(re_train_img_cnt, re_train_face_cnt, re_train_eye_cnt)
                        self.rdThread.start()
                        
                        re_img_cnt = 0
                        re_face_cnt = 0
                        re_eye_cnt = 0
                        re_train_img_cnt = []
                        re_train_face_cnt = []
                        re_train_eye_cnt = []
                        self.rd_flag = False 
                    
            now = time.time()
            
            # 재학습하는 경우 2초마다 count 정보 추가
            if self.rd_flag and now - start_time >= 2:
                re_train_img_cnt.append(re_img_cnt)
                re_train_face_cnt.append(re_face_cnt)
                re_train_eye_cnt.append(re_eye_cnt)
                
                re_img_cnt = 0
                re_face_cnt = 0
                re_eye_cnt = 0
                start_time = now
            
            # 5초마다 데이터 stack
            if now - start_time >= 5 :
                self.sThread.setInfo(self.img_count, self.face_count, self.eye_count, deepcopy(self.confidence), deepcopy(self.coord), now)
                self.change_conf_lst.append(self.mean(self.confidence))
                
                self.img_count = 0
                self.face_count = 0
                self.eye_count = 0
                self.confidence = []
                self.coord = []
                
                start_time = now
                append_cnt +=1
                
                # 재학습 여부 판단
                conf_mean = self.mean(self.change_conf_lst)
                if conf_mean >= 60 and (not self.rd_flag) :
                    self.change_cnt += 1
                        
                    if self.change_cnt == 12 :
                        self.yml_ver += 1
                        self.change_cnt = 0 
                            
                        dir_lst = os.listdir("./data/register/registered_info")
                        yml = "train" + str(self.yml_ver) + ".yml"
                            
                        if yml in dir_lst :
                            self.recognizer.read("./data/register/registered_info/" + yml)
                        else : 
                            self.rd_flag = True

                elif conf_mean < 20 and (not self.rd_flag) :
                    self.change_cnt = 0
                
                # 저장
                if append_cnt == 12 :
                    self.sThread.setIndex(self.global_idx)
                    self.sThread.start()
                    self.global_idx += 1
                    append_cnt = 0
                       
        self.msg.setText("ready for detection")
        self.imgs = []
        self.img_count = 0
        self.face_count = 0
        self.eye_count = 0

    def detect(self, gray, frame):
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        face = self.size_filter(faces)
        roi_gray = []

        for (x,y,w,h) in face:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)

            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]

            eyes = self.eyes_cascade.detectMultiScale(roi_gray, 1.1, 3)
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex,ey), (ex+ew, ey+eh), (0, 255, 0), 2)
                    
        if len(faces) == 0 :
            flag = [False, False]
            eyes = []
                
        elif len(faces) > 0 and len(eyes) == 0 :
            flag = [True, False]
                
        else : 
            flag = [True, True]
            
        coord = [faces, eyes]
        return flag, roi_gray, coord 
    
    def size_filter(self, coords) :
        if len(coords) == 0 :
            return []
        
        size = 0 
        best_coord = np.array([0,0,0,0])

        for coord in coords :
            now = coord[2]*coord[3]
            if now > size :
                size = now
                best_coord = coord

        return [best_coord]
    
class saveThread(QThread) :
    def __init__(self):
        QThread.__init__(self)
        self.idx = -1
        
        self.img_count = []
        self.face_count = []
        self.eye_count = []
        self.confidence = []
        self.coord = []
        self.time = []
        
    def __del__(self):
        self.img = []
        self.wait()
        
    def setInfo(self, c_img, c_face, c_eye, conf, coord, now) :
        self.img_count.append(c_img)
        self.face_count.append(c_face)
        self.eye_count.append(c_eye)
        
        mean = self.mean(conf)
        self.confidence.append(mean)

        x = self.mean([c[0] for c in coord])
        y = self.mean([c[1] for c in coord])
        wh = self.mean([c[2] for c in coord])
        self.coord.append([x, y, wh, wh])
        
        self.time.append(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(now)))
        
        print("-"*90)
        print("[{}] : values for aggreagation are added --length {}".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time())), len(self.img_count)))
        print("{}, {}, {}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}".format(c_img, c_face, c_eye, mean, x, y, wh, wh))
        print("-"*90)
        
    def setIndex(self, idx) :
        self.idx = idx
        
    def setToken(self, token) :
        self.token = token
    
    def run(self):
        self.save()
        
    def mean(self, lst) :
        if len(lst) == 0 :
            return 0
        
        return sum(lst) / len(lst)
        
    def save(self) :
        x = [c[0] for c in self.coord]
        y = [c[1] for c in self.coord]
        wh = [c[2] for c in self.coord]
        
        with open("./data/detect/detected_info/detected_data{}.txt".format(self.idx), "w") as f:
            contents =  ""
            idx = len(self.confidence)
            
            for img, face, eye, conf, x, y, wh, t in zip(self.img_count, self.face_count, self.eye_count, self.confidence, x, y, wh, self.time): 
                contents += "{}, {}, {}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {}".format(img, face, eye, conf, x, y, wh, wh,t)
                
                idx -= 1
                if idx != 0 :  
                    contents += "\n"
            f.write(contents)
            
        data = {"msg" : contents, "token" : self.token}
        result = requests.post(KAFKA_SERVER_IP, data=data)
            
        self.img_count = []
        self.face_count = []
        self.eye_count = []
        self.confidence = []
        self.coord = []
        self.time = []
            
        print("-"*90)
        print("[{}] : saving is finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("-"*90)
        
class reg_detThread(QThread) :
    def __init__(self):
        QThread.__init__(self)
        self.frame = []
        self.coords = []
        self.eye = 0
        self.data_size = 0
        self.ver = -1
        
    def __del__(self) : 
        self.wait()
    
    def add(self, detected_frame, coords) :
        self.frame.append(detected_frame)
        self.coords.append(coords)
        self.data_size += 1
    
    def setVersion(self, ver) :
        self.ver = ver
    
    def setDThread(self, dThread) :
        self.dThread = dThread
    
    def setCnt(self, img, face, eye) :
        cnt = [(str(i), str(f), str(c)) for i, f, c in zip(img, face, eye)]
        self.cnt = [", ".join(x) for x in cnt]
        
    def run(self):
        self.register_with_detect()

    def register_with_detect(self) :
        for idx, img in enumerate(self.frame[:100]) :
            cv2.imwrite("./data/register/registered_png/register_{}.png".format(idx), img, [])
        
        labels = self.make_labels(self.frame[:100])
        
        recognizer = cv2.face.LBPHFaceRecognizer_create()
        recognizer.train(self.frame[:100], labels)
        recognizer.save("./data/register/registered_info/train" + str(self.ver) +".yml")
        recognizer.read("./data/register/registered_info/train" + str(self.ver) +".yml")
        
        self.coord = self.text_rectangle(self.coords)
        with open("./data/register/registered_info/rectangle_coord.txt", "w") as f:
            f.write("\n".join(self.coord))
            
        with open("./data/register/registered_info/count.txt", "w") as f:
            f.write("\n".join(self.cnt))
            
        with open("./data/register/registered_info/train_datetime.txt", "w") as f:
            f.write(str(time.strftime("%Y/%m/%d %H:%M", time.localtime(time.time()))))
            
        confidence = []
        contents=""
        for frame in self.frame[100:] :
            confidence.append(recognizer.predict(frame)[1])
            
        with open("./data/register/registered_info/confidence.txt", "w") as f:
            f.write("\n".join([str(conf) for conf in confidence]))
            
        self.frame = []
        self.coords = []
        self.eye = 0
        self.ver = -1
        self.data_size = 0
        
        self.dThread.recognizer = recognizer
        print("-"*90)
        print("[{}] : re-train finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("-"*90)
        
        self.quit()
    
    def mean(self, lst) :
        if len(lst) == 0 :
            return 0
        
        return sum(lst) / len(lst)
        
    def text_rectangle(self, coords) :
        result = []
        for lst in coords :
            result.append(", ".join([str(x) for x in lst]))
            
        return result
        
    def make_labels(self, images) :
        labels = []

        length = len(images)
        for idx in range(length) :
            labels.append(1)

        return np.array(labels)
        
    def making_gray(self, images) :
        result = []

        for image in images :
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            result.append(gray)

###############################################################################################################

    
###############################################################################################################
class openRegisterThread(QThread) : 
    def __init__(self, params, widget):
        QThread.__init__(self)
        
        self.widget = widget
        self.setParams(params)
        self.params = params

    def __del__(self):
        self.wait()
        
    def setParams(self, params) :
        self.video_capture = params[0]
        self.face_cascade = params[1]
        self.eyes_cascade = params[2]
        self.msg = params[3]
        self.interval = params[4]
        self.flag_while = params[5]

    def run(self):
        self.get_size()
    
    def get_size(self) :
        read, frame = self.video_capture.read()
        h,w,c = HWC
        
        self.widget.dialog_size = (w,h)
        with open("./data/register/registered_info/cam_size.txt", "w") as f:
            f.write(str(w) + ", " + str(h))
            
        self.quit()

class registerDialog(QDialog):
    def __init__(self, params, size, widget) :
        QDialog.__init__(self)  
        self.params = params
        self.setWindowTitle('Register')
        self.resize(size[0], size[1])
        self.widget = widget
        
        self.start = False
        
        self.registerThread = registerThread()
        self.registerThread.setParams(params)
        self.registerThread.setDialog(self)

        self.imageLabel = QLabel()
        self.RunButton = QPushButton('Start')
        self.ExitButton = QPushButton('Stop and Exit')

        self.RunButton.clicked.connect(self.runClicked)
        self.ExitButton.clicked.connect(self.stopClicked)

        self.layout = QVBoxLayout()
        self.layout.addWidget(self.imageLabel)
        self.layout.addWidget(self.RunButton)
        self.layout.addWidget(self.ExitButton)
        
        self.registerThread.setImageLabel(self.imageLabel)
        self.setLayout(self.layout)
        
        self.registerThread.start()
        self.start = False
        
    def setParams(self, params) : 
        self.video_capture = params[0]
        self.face_cascade = params[1]
        self.eyes_cascade = params[2]
        self.msg = params[3]
        self.interval = params[4]
        self.flag_while = params[5]
        
        self.registerThread.setParams(params)
        
    def runClicked(self) :
        if not self.start :
            print("#"*60)
            print("[{}] : start register".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
            print("#"*60, "\n")
            print("[{}] : step1 start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))

            self.registerThread.start_register = True
            self.registerThread.start_saving = False
            self.registerThread.finish_saving = [False]
            self.registerThread.detected_lst = []
            self.registerThread.coord_lst = []
            self.registerThread.confidence_lst = []
            self.registerThread.eye_count = 0
            self.start = True
        
    def stopClicked(self) :
        if self.start : 
            print("#"*60)
            print("[{}] : stop register".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
            print("#"*60, "\n")

            self.registerThread.start_register = False
            self.registerThread.start_saving = False
            self.registerThread.finish_saving = [False]
            self.registerThread.detected_lst = []
            self.registerThread.coord_lst = []
            self.registerThread.confidence_lst = []
            self.registerThread.eye_count = 0
            self.start = False
            
    def closeEvent(self, event):
        self.registerThread.go_flag = False
        self.registerThread.quit()
        self.widget.isDialog = False
        
        print("#"*60)
        print("[{}] : success to register".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("#"*60, "\n")
                
        event.accept()
###############################################################################################################


###############################################################################################################
class registerThread(QThread) :
    def __init__(self):
        QThread.__init__(self)
        self.imgs = []
        self.go_flag = True
        self.start_register = False
        self.coord_lst = []
        self.detected_lst = []
        self.confidence_lst = []
        
        self.img_count = 0
        self.face_count = 0
        self.eye_count = 0
        self.count = []
        
        self.step = [1]
        self.start_saving = False
        self.finish_saving = [False]
        
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()
        
    def __del__(self):
        self.wait()
        self.go_flag = False
        
    def setParams(self, params) :
        self.video_capture = params[0]
        self.face_cascade = params[1]
        self.eyes_cascade = params[2]
        self.msg = params[3]
        self.interval = params[4]
        self.flag_while = params[5]
        
    def setDialog(self, dialog) :
        self.dialog = dialog
        
    def setImageLabel(self, imageLabel) :
        self.imageLabel = imageLabel
        self.imageLabel.setAlignment(Qt.AlignCenter)
        self.imageLabel.setScaledContents(True)
        self.imageLabel.setMinimumSize(1,1)

    def run(self):
        self.face_register()
    
    def face_register(self) :
        start_time = 0
        font = cv2.FONT_HERSHEY_PLAIN
        color = (0,0,0)
        stroke = 2
       
        while self.go_flag :
            if self.start_register :
                read, frame = self.video_capture.read()
                frame = cv2.resize(frame, WH)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                h, w, c = HWC
                end_time = time.time()

                # step1 : 얼굴이 인식되는 장소와 bounding box를 찾기
                # 요청 : 정면의 카메라를 보고 있어주세요. (촬영횟수 count)
                if self.step[0] == 1 :
                    flag, detected_frame, coords, eye = self.detect(gray, frame, (255,0,0))
                    cv2.circle(frame, (w//2, h//2), 3, (255,0,0), -1)
                    
                    if len(coords) != 0 :
                        check = self.check_include_center(coords[0], w, h)
                    else :
                        check = False
                        
                    if flag and check:
                        name = "place at center of screen, complete {:.1f}%".format(len(self.coord_lst)/50*100)
                        cv2.putText(frame, name, (20, 20), font, 1,(255,0,0), stroke, cv2.LINE_AA)
                        self.coord_lst.append(coords[0])

                        if len(self.coord_lst) == 50 :
                            print("[{}] : step1 finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                            print("[{}] : step2 start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                            self.set_rectangle(self.coord_lst)
                            self.step[0] += 1

                            start_time = time.time()

                    elif flag and not check:
                        name = "place at center of screen, complete {:.1f}%".format(len(self.coord_lst)/50*100)
                        cv2.putText(frame, name, (20, 20), font, 1,(255,0,0), stroke, cv2.LINE_AA)

                    elif not flag:
                        name = "fail to detect face,       complete {:.1f}%".format(len(self.coord_lst)/50*100)
                        cv2.putText(frame, name, (20, 20), font, 1,(255,0,0), stroke, cv2.LINE_AA)

                # step2 : 얼굴이 인식되는 장소에 bounding box를 그리고 얼굴을 인식하여 사진 모으기(50~100)
                # 요청  : 정면의 카메라를 보고 있어주세요. (촬영횟수 count)
                #         만약 사각형이 얼굴을 찾지 못한다면, 다시 시작해주세요.
                elif self.step[0] == 2 :                       
                    flag, detected_frame, coords, eye = self.detect(gray, frame, (0,0,255))
                    self.img_count += 1
                    
                    if len(coords) != 0 :
                        self.face_count += 1
                        area, check = self.check_overlap(coords[0])
                    else :
                        area = 0
                        check = False
                        

                    if end_time - start_time >= 5 and  check:
                        detected_frame = cv2.cvtColor(detected_frame, cv2.COLOR_RGB2BGR)
                        self.detected_lst.append(detected_frame)
                        
                        if eye : 
                            self.eye_count += 1
                        
                    name = "mathcing 80% rectangle,    complete {:.1f}%".format(len(self.detected_lst)/100*100)
                    cv2.putText(frame, name, (20, 20), font, 1,(255,0,0), stroke, cv2.LINE_AA)
                    cv2.rectangle(frame, (self.x, self.y), (self.x+self.w, self.y+self.h), (255,0,0), 2)

                    if self.img_count == 50 :
                        self.count.append((str(self.img_count), str(self.face_count), str(self.eye_count)))
                        self.img_count = 0
                        self.face_count = 0
                        self.eye_count = 0
                    
                    if len(self.detected_lst) == 100 :
                        print("[{}] : step2 finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        print("[{}] : step3 start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        self.step[0] += 1

                # step3 : 저장
                elif self.step[0] == 3 :
                    if not self.start_saving :
                        tThread = trainThread(self.detected_lst, self.count)
                        tThread.setStep(self.step)
                        tThread.setFlag(self.finish_saving)
                        tThread.setRecognizer(self.recognizer)
                        tThread.setCoord(self.coord_lst)
                        tThread.start()
                        self.start_saving = True
                        
                    if self.finish_saving[0] :
                        self.step[0] += 1
                        start_time = time.time()
                        self.recognizer.read("./data/register/registered_info/train1.yml")
                        
                        print("[{}] : step3 finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        print("[{}] : step4 start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        
                    name = "saving register result... wait please"
                    cv2.putText(frame, name, (20, 20), font, 1,(255,0,0), stroke, cv2.LINE_AA)
                    cv2.rectangle(frame, (self.x, self.y), (self.x+self.w, self.y+self.h), (255,0,0), 2)
                    
                # step4 : confidence 확인
                elif self.step[0] == 4 : 
                    flag, detected_frame, coords, eye = self.detect(gray, frame, (0,0,255))                    
                    
                    if len(coords) != 0 :
                        coord = coords[0]
                        area, check = self.check_overlap(coord)
                    else :
                        area = 0
                        check = False
                        
                    if end_time - start_time >= 5 and  check:
                        roi_gray = gray[coord[1]:coord[1]+coord[3], coord[0]:coord[0]+coord[2]]
                        label, confidence = self.recognizer.predict(roi_gray)
                        self.confidence_lst.append(confidence)
                        
                    name = "mathcing 80% rectangle,    complete {:.1f}%".format(len(self.confidence_lst)/100*100)
                    cv2.putText(frame, name, (20, 20), font, 1,(255,0,0), stroke, cv2.LINE_AA)

                    coord = self.size_filter(coords)
                    cv2.rectangle(frame, (self.x, self.y), (self.x+self.w, self.y+self.h), (255,0,0), 2)

                    if len(self.confidence_lst) == 100 :
                        cThread = confidenceThread(self.confidence_lst)
                        cThread.start()
                        self.start_saving = True
                        
                        print("[{}] : step4 finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        print("")
                        
                        self.start_register = False
                        self.dialog.start = False
                    
            else :
                self.step = [1]
                read, frame = self.video_capture.read()
                frame = cv2.resize(frame, WH)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                h, w, c = HWC 
            
                name = "ready for register,place at center of sreen"
                cv2.putText(frame, name, (20, 20), font, 1,(255,0,0), stroke, cv2.LINE_AA)
                cv2.circle(frame, (w//2, h//2), 3, (255,0,0), -1)

            q_img = QImage(frame, w, h, w*c, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(q_img)
            pixmap_image = QPixmap(pixmap)

            self.imageLabel.setPixmap(pixmap_image)
            
                
    def check_include_center(self, coord, w, h) :
        if (coord[0] < (w//2) < (coord[0]+coord[2])) and  (coord[1] < (h//2) < (coord[1]+coord[3])) :
            return True
        else :
            return False

    def check_overlap(self, coord) :
        r1 = (self.x , self.y-self.h, self.x+self.w, self.y)
        r2 = (coord[0] , coord[1]-coord[3], coord[0]+coord[2], coord[1])
        
        dx = min(r1[2], r2[2]) - max(r1[0], r2[0])
        dy = min(r1[3], r2[3]) - max(r1[1], r2[1])
        area = dx*dy
        
        if (dx>=0) and (dy>=0) and (area/(self.w*self.h) >= 0.8) :
            return area/self.w*self.h, True
        else :
            if dx*dy<= 0 :
                area = 0
            return 0, False 
                
    def detect(self, gray, frame, color):
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        face = self.size_filter(faces)
        roi_color = []
        eye = False
        
        for (x,y,w,h) in face : 
            cv2.rectangle(frame, (x,y), (x+w, y+h), color, 2)

            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            
            eyes = self.eyes_cascade.detectMultiScale(roi_gray, 1.1, 3)
            if len(eyes) == 0:
                eye = False
            else :
                eye = True
                    
        if len(face) == 0 :
            flag = False
        else :
            flag = True
            
        coord = face
        return flag, roi_color, coord , eye
        
    def set_rectangle(self, coord_lst) :
        x_lst = [coord[0] for coord in coord_lst]
        y_lst = [coord[1] for coord in coord_lst]
        w_lst = [coord[2] for coord in coord_lst]
        h_lst = [coord[3] for coord in coord_lst]

        x = int(sum(x_lst)/len(x_lst))
        y = int(sum(y_lst)/len(y_lst))
        w = int(sum(w_lst)/len(w_lst))
        h = int(sum(h_lst)/len(h_lst))
        
        self.x = x
        self.y = y
        self.w = w
        self.h = h 
        
    def size_filter(self, coords) :
        if len(coords) == 0 :
            return []
        
        size = 0 
        for coord in coords :
            now = coord[2]*coord[3]
            if now > size :
                size = now
                best_coord = coord

        return [best_coord]
    
class trainThread(QThread) :
    def __init__(self, imgs, count):
        QThread.__init__(self)
        self.imgs = imgs
        self.count = count
        
    def __del__(self):
        self.wait()

    def run(self):
        self.save_and_train()
        
    def setRecognizer(self, recognizer) :
        self.recognizer = recognizer
        
    def setStep(self, step) :
        self.step = step
    
    def setFlag(self, flag) :
        self.finish_saving = flag
    
    def setCoord(self, coord) :
        self.coord = coord
        
    def text_rectangle(self, coords) :
        result = []
        for lst in coords :
            result.append(", ".join([str(x) for x in lst]))
            
        return result
        
    def save_and_train(self) :
        for idx, img in enumerate(self.imgs) :
            cv2.imwrite("./data/register/registered_png/register_{}.png".format(idx), img, [])
        
        labels = self.make_labels(self.imgs)
        grays = self.making_gray(self.imgs)
        
        self.recognizer.train(grays[10:], labels[10:])
        self.recognizer.save("./data/register/registered_info/train1.yml")
        
        coord = self.text_rectangle(self.coord)
        with open("./data/register/registered_info/rectangle_coord.txt", "w") as f:
            f.write("\n".join(coord))
            
        cnt = [", ".join(x) for x in self.count]
        with open("./data/register/registered_info/count.txt", "w") as f:
            f.write("\n".join(cnt))
        
        self.finish_saving[0] = True
        self.quit()
        
    def make_labels(self, images) :
        labels = []

        length = len(images)
        for idx in range(length) :
            labels.append(1)

        return np.array(labels)
        
    def making_gray(self, images) :
        result = []

        for image in images :
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            result.append(gray)

        return result
    
class confidenceThread(QThread) :
    def __init__(self, confidence):
        QThread.__init__(self)
        self.confidence = confidence
        
    def __del__(self):
        self.wait()

    def run(self):
        self.save()
        
    def save(self) :
        with open("./data/register/registered_info/confidence.txt", "w") as f:
            f.write("\n".join([str(conf) for conf in self.confidence]))
            
        with open("./data/register/registered_info/train_datetime.txt", "w") as f:
            f.write(str(time.strftime("%Y/%m/%d %H:%M", time.localtime(time.time()))))

        print("-"*90)
        print("[{}] : train and saving are finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("-"*90)
        print("")
        
        self.quit()
###############################################################################################################

In [7]:
class mainWindow(QMainWindow) :
    def __init__(self) :
        QMainWindow.__init__(self)
        self.setWindowTitle("PyQt & openCV demo3")
        self.resize(300,200)
        
        self.main_widget = BeforeLogin()
        self.setCentralWidget(self.main_widget)
        
    def closeEvent(self, event) :
        reply = QMessageBox.question(self, 'Message',"Are you sure to quit?", 
                                           QMessageBox.Yes, QMessageBox.No)

        if reply == QMessageBox.Yes:
            if self.main_widget.token :
                self.start = False
                
                if self.main_widget.afterLoginWidget.isDialog :
                    self.main_widget.afterLoginWidget.registerDialog.close()
                
                self.main_widget.afterLoginWidget.video_capture.release()
                self.main_widget.afterLoginWidget.detectionThread.quit()
            event.accept()
        else:
            event.ignore()

        
        event.accept()
        
def main():
    app = QApplication([])
    main_window = mainWindow()
    main_window.show()
    
    try :
        sys.exit(app.exec_())
        
        print("#"*60)
        print("[{}] : (1) program stop".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("#"*60, "\n")
        
    except :
        print("")
        print("#"*60)
        print("[{}] : (2) program stop".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("#"*60, "\n")
        
if __name__ == '__main__':
    main()

############################################################
[2019/01/24 16:04:27] : program start
############################################################ 

############################################################
[2019/01/24 16:04:30] : 'student100' try to login
[2019/01/24 16:04:30] : 'student100' success to login
############################################################ 

############################################################
[2019/01/24 16:04:31] : camera is conntected
############################################################ 

############################################################
[2019/01/24 16:04:32] : fail to register -- ready for register
############################################################ 

############################################################
[2019/01/24 16:04:34] : try to register
dialog size has (480, 360)
############################################################ 

############################################################
[2